In [1]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.13.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0



11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1839 - sparse_categorical_accuracy: 0.6502
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 11ms/step - loss: 1.1847 - sparse_categorical_accuracy: 0.6510 - val_loss: 0.7321 - val_sparse_categorical_accuracy: 0.7790
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4486 - sparse_categorical_accuracy: 0.8781
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4440 - sparse_categorical_accuracy: 0.8790 - val_loss: 0.5283 - val_sparse_categorical_accuracy: 0.8400
Epoch 3/10
16/32 [==============>...............] - ETA: 0s - loss: 0.2716 - sparse_categorical_accuracy: 0.9277
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2966 - sparse_categorical_accuracy: 0.9210 - val_loss: 0.4758 - val_sparse_categorical_accuracy: 0.8530
Epoch 4/10
26/32 [=========

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3463 - sparse_categorical_accuracy: 0.1290 - 121ms/epoch - 4ms/step
Untrained model, accuracy: 12.90%


In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.3892 - sparse_categorical_accuracy: 0.8770 - 53ms/epoch - 2ms/step
Restored model, accuracy: 87.70%


In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [10]:
os.listdir(checkpoint_dir)

['cp-0025.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0040.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001']

In [11]:

latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4801 - sparse_categorical_accuracy: 0.8750 - 120ms/epoch - 4ms/step
Restored model, accuracy: 87.50%


In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4801 - sparse_categorical_accuracy: 0.8750 - 161ms/epoch - 5ms/step
Restored model, accuracy: 87.50%


In [14]:
#using .keras

# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1739 - sparse_categorical_accuracy: 0.6630
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4165 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2840 - sparse_categorical_accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2163 - sparse_categorical_accuracy: 0.9400
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1487 - sparse_categorical_accuracy: 0.9630


In [15]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)


32/32 - 0s - loss: 0.4590 - sparse_categorical_accuracy: 0.8550 - 128ms/epoch - 4ms/step
Restored model, accuracy: 85.50%
32/32 [==============================] - 0s 1ms/step
(1000, 10)


In [17]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5


32/32 [==============================] - 1s 4ms/step - loss: 1.1756 - sparse_categorical_accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4311 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2976 - sparse_categorical_accuracy: 0.9160
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2280 - sparse_categorical_accuracy: 0.9350
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1649 - sparse_categorical_accuracy: 0.9600


In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1695 - sparse_categorical_accuracy: 0.6490
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4145 - sparse_categorical_accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2799 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1988 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1454 - sparse_categorical_accuracy: 0.9730


In [21]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4155 - sparse_categorical_accuracy: 0.8600 - 169ms/epoch - 5ms/step
Restored model, accuracy: 86.00%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


In [23]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1535 - sparse_categorical_accuracy: 0.6710
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4258 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2866 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2089 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1560 - sparse_categorical_accuracy: 0.9600


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               401920    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4308 - sparse_categorical_accuracy: 0.8550 - 117ms/epoch - 4ms/step
Restored model, accuracy: 85.50%


In [26]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


32/32 - 0s - loss: 0.4308 - sparse_categorical_accuracy: 0.8550 - 53ms/epoch - 2ms/step
Restored model, accuracy: 85.50%
